In [ ]:
#
# Preliminaries
#

#
# some important Imports...
#
import numpy as np
from ase import Atoms
from ase.io import read, write
from ase.visualize import view
import matplotlib.pyplot as plt
import nglview as nv
from ase.build import molecule

%load_ext aiida
%aiida
from aiida import orm

#
# and definitions of visualization functions (see last exercises)
#
def view_structure(structure,myvec=[]):
    t = nv.ASEStructure(structure)
    w = nv.NGLWidget(t, gui=True)
    w.add_unitcell()
    w.add_ball_and_stick()
    w.add_representation('label',label_type='atomindex',color='black')
    w.add_representation('spacefill',selection=myvec,color="blue",radius=0.5)
    return w

def view_trajectory(trajectory,myvec=[]):
    t2 = nv.ASETrajectory(trajectory)
    w2 = nv.NGLWidget(t2, gui=True)
    #w2.add_unitcell()
    w2.add_ball_and_stick()
#    w2.add_representation('spacefill',selection=myvec,color="blue",radius=0.5)
    return w2

# Exercise 7: the dehydration of ethanol
Date: Wednesday 2 April 2025

**Submission deadline**: End of April 2025

In this exercise we will investigate a very important chemical reaction, namely the production of ethene (ethylene) from ethanol. Ethanol is heated with an excess of concentrated sulphuric acid at a temperature of 170°C. The gases produced are passed through sodium hydroxide solution to remove the carbon dioxide and sulphur dioxide produced from side reactions. Another way to favor this reaction is in presence of a catalyst surface, which makes the reaction exothermic even at room temperature, and the free energy barrier for the reaction is highly reduced, so to obtain reasonable reaction rates. The use of hematite (an iron-based nanocrystal) as a catalyst is described computationally in [this paper (clickable link)](https://pubs.rsc.org/en/content/articlelanding/2016/RA/C6RA08509A#!divAbstract) by Lopes *et. al.*. We are interested in the gas phase reaction, in particular in the reactants and the products energy, to estimate the “energy of reaction”.

The reaction is the following:

\begin{equation}
    \text{CH}_3\text{CH}_2\text{OH} \longrightarrow \text{C}_2\text{H}_4 + \text{H}_2\text{O}
\end{equation}

and, in the gas phase, the equation looks like

![The dehydration of **(a)** ethanol after overcoming a **(b) transition state (TS)** into its **(c)** products: ethene and water.](ethanol.png "Figure 1")

In the supplementary material of the paper, which can be found [here](http://www.rsc.org/suppdata/c6/ra/c6ra08509a/c6ra08509a1.pdf), several calculations with different methods ranging from DFT to hybrid methods to even higher-level methods are described, together with the basis set dependence of the results. We will compare our results with those published by performing calculations within the DFT framework for two different functionals: the PBE parametrization of the generalized gradient approximation (GGA), introduced in Exercise 7, and the hybrid functional B3LYP (named after those who developed it: Becke, 3-parameter, Lee–Yang–Parr); within each functional we will optimize the geometry at two-three basis sets per functional, thereby obtaining a final (hopefully converged!) value for the total inner energy $U$. From here we will use thermodynamics to relate $U$ to the Gibbs free energy $G$; see the Figure below:

<img src="rxn-coord.png" alt="isolated" width="700"/>

# 0. Calculations with PBE

The first thing we will do is a set of reference calculations in which we will optimize the reactants and the products with **AiiDAlab** and the transition state with cp2k "by hand".

## 0.1 Optimizing ethanol

We note that in the exercise directory we have four .xyz files:



In [ ]:
!ls *.xyz

We upload the first file, **ethanol.xyz** to the AiiDAlab database:

In [ ]:
#
# After those commands, the structure is available in the database for successive geometry optimization
#
asegeo = read("ethanol.xyz")
asegeo.center()
aiidastructure = orm.StructureData(ase=asegeo)
aiidastructure.store()
view_structure(asegeo)

In [ ]:
#
# After those commands, the structure is available in the database for successive geometry optimization
#
asegeo = read("H2O.xyz")
asegeo.center()
aiidastructure = orm.StructureData(ase=asegeo)
aiidastructure.store()
view_structure(asegeo)

### Launch the geometry optimization with AiiDAlab. Check the simulation using the usual commands:

        verdi process list -a
        verdi calcjob gotocomputer ...

When the simulation is completed, mark the pk of the optimization, behind it in the output of verdi process list -a there is the "Cp2kCalculation" 

In [ ]:
!verdi process list -a






In [ ]:
!verdi process show xxxx
!verdi calcjob inputcat yyyy


Among the information that we can infer from the input file, we note that this simulation has been performed with a good basis set, **TZV2P-MOLOPT-GTH** We will perform all the optimizations with this basis set, and then test the energy (with fixed nuclei) with different functional and basis sets. 

### Load the optimized structure and save it in the output file

In [ ]:
ase_structure = {}
output_number = {}

In [ ]:
ase_structure = {}
output_number = {}
molecule = "ethanol"
#
# Note the pk of the equilibrium geometry (also seen as "output_structure")
#
output_number[molecule] = INSERT IT HERE
ase_structure[molecule] = load_node(output_number[molecule]).get_ase()
ase_structure[molecule].write(molecule+"/"+molecule+"_opt.xyz")
myview=view_structure(ase_structure[molecule])
myview.add_distance(atom_pair=[[0,8]], label_color="black")
myview.add_distance(atom_pair=[[0,3]], label_color="black")
myview

In [ ]:
molecule ="ethanol"
#
# Check that the xyz has been saved
#
!ls {molecule}

In [ ]:

molecule = "H2O"
#
# Note the pk of the equilibrium geometry (also seen as "output_structure")
#
output_number[molecule] = INSERT IT HERE
ase_structure[molecule] = load_node(output_number[molecule]).get_ase()
ase_structure[molecule].write(molecule+"/"+molecule+"_opt.xyz")
myview=view_structure(ase_structure[molecule])
myview.add_distance(atom_pair=[[0,1]], label_color="black")
myview.add_distance(atom_pair=[[1,2]], label_color="black")
myview

In [ ]:
molecule = 'H2O'
file = molecule+"/PBE/SZV/"+"NAME OF THE CUBE FILE"
atoms = read(file)
a=!head -2 {file} | tail -1
b = str(a)

view_homo=nv.NGLWidget()
caption_homo = molecule+" HOMO"
view_homo.add_component(nv.ASEStructure(atoms))
c_2 = view_homo.add_component(file)
c_2.clear()
c_2.add_surface(color='blue', isolevelType="value", isolevel=-0.01, opacity=0.05)
c_3 = view_homo.add_component(file)
c_3.clear()
c_3.add_surface(color='red', isolevelType="value", isolevel=0.01, opacity=0.05)


#
# And finally the visualization itself
#

print (caption_homo)
view_homo

In [ ]:
molecule = 'H2O'
file = molecule+"/PBE/SZV/"+"NAME OF THE CUBE FILE"
atoms = read(file)
a=!head -2 {file} | tail -1
b = str(a)

view_lumo=nv.NGLWidget()
caption_lumo = molecule+" LUMO"
view_lumo.add_component(nv.ASEStructure(atoms))
c_2 = view_lumo.add_component(file)
c_2.clear()
c_2.add_surface(color='blue', isolevelType="value", isolevel=-0.01, opacity=0.05)
c_3 = view_lumo.add_component(file)
c_3.clear()
c_3.add_surface(color='red', isolevelType="value", isolevel=0.01, opacity=0.05)


#
# And finally the visualization itself
#

print (caption_lumo)
view_lumo

## Repeat the procedure with H2O.xyz and ethene.xyz

Remember to save the best configurations in H2O_opt.xyz and ethene_opt.xyz


# Transition state

For the transition state we will run the dimer method starting from a pretty close structure to the true transition state. We will use the file ts.xyz as initial structure, and run the commands in the terminal (**from the Exercise_6 directory**)

    cd ts_OPT 
    cp2k.psmp -i ts_dimer.inp > ts_dimer.out &
   
   
This will reach the transition state using the "dimer method" discussed in the lecture. 

After the simulation is finished (this can be checked with:

    tail -f ts_dimer.out # interrupted with CTRL+C

) 

we visualize the trajectory of optimization here:

In [ ]:
TRAJ_TS = './ts_OPT/ts-pos-1.xyz'
traj_TS = read(TRAJ_TS, index=":")

myview=view_trajectory(traj_TS)
myview.add_distance(atom_pair=[[0,8]], label_color="black")    
myview.add_distance(atom_pair=[[0,3]], label_color="black")  
#
# Note these additional methods to add some distances in the plot
#
myview

In [ ]:
final=traj_TS[len(traj_TS)-1]
#
# Export the final and best trajectory 
#
final.write('ts/ts_opt.xyz')

INI_TS = 'ts_opt.xyz'
ini_TS = read(INI_TS)

myview=view_structure(ini_TS)
myview.add_distance(atom_pair=[[0,8]], label_color="black")    

myview

# Computing the reaction energies at different theory levels

In the current `Exercise_07` directory you will find the following four subdirectories for the reactants, product and transition states. Inside those directories you will find eight files:

* `H2O\_template.inp}` for water energy runs and the corresponding `H2O_opt.xyz` file providing the geometry;
* `ethanol\_template.inp` for the energy runs of ethanol and the corresponding `ethanol_opt.xyz` file providing the geometry;
* `ethylene\_template.inp` for the energy runs of ethylene and the corresponding `ethylene_opt.xyz` file providing the geometry;
* `ts\_template.inp` for the transition state and the corresponding `ts_opt.xyz` file providing the geometry`

Move into the `PBE` directory and make a new subdirectory `SZV`, the name of the first (and simplest) basis sets we will examine today, and move into the `SZV` directory:

```bash
$ cd PBE
$ mkdir SZV
$ cd SZV
```

Copy the `H2O_template.inp` file from the parent directory into the current one and rename it:

```bash
$ cp ../../H2O_template.inp H2O.inp
```

*NOTE THAT THE XC FUNCTIONAL DIRECTIVES ARE INCLUDED AS A pbe.inc FILE (LOOK FOR IT IN THE INPUT FILE!!!)*

Submit the calculation on H2O in the current directory using the terminal
```
cp2k.psmp -i H2O.inp > H2O.out &
```

For the `H2O`, `ethanol`, and `ethene` projects, notice how the `RUN_TYPE` keyword in the `&GLOBAL` section of the input file is associated with `ENERGY`. This means (unsurprisingly) that we will not be optimizing the geometry, as we've done in previous exercises. While the calculation is running, open a new Terminal in your launcher and visualize the the `H2O.inp` file with vim.

The structure and layout of the input files for `ethanol`, `ethene` and `transition states` ..).

Proceed to the first assignment.

## 1. Calculations with PBE

#### Assignment 1:The remaining simulations for the SVZ basis set
Repeat and submit the remaining calculations for `ethanol.inp`, `ethene.inp`, and `ts.inp` for the PBE.

In [ ]:
## -- your code here

#### End Assignment 1

#### Assignment 2: Visualization of the  orbitals
Visualize the structures and trajectories of the reactant ethanol as well as of the products ethene and water. Remember that the trajectory will have been generated as an output file in the following nominal format: `project_name-pos-1.xyz`.

The three energy runs produce one `.cube` file for HOMO and one for LUMO and they are named according to `project_name-WFN_orbital_number_1-1_0.cube`; use code from exercise 4 to visualize the HOMO and LUMO orbitals of the optimized geometries of all three molecules.

In [ ]:
## -- Your code here

#### End Assignment 2

## 2. Calculations with B3LYP

Once the above calculations have concluded, we want to perform the identical calculations, except using B3LYP as a functional. B3LYP is what is known as a **hybrid (exchange-correlation) functional** because the separate exchange $X$ and correlation $C$ effects are each defined as a mixture ('hybrid') between the exact exchange from Hartree-Fock and that from a GGA functional. The formula for the B3LYP xc-functional is

\begin{equation}
    E_{XC}^{B3LYP} = E_{X}^{LDA} + a_0(E_{X}^{HF}-E_{X}^{LDA}) + a_X(E_{X}^{GGA}-E_{X}^{LDA}) + E_{C}^{LDA} + a_C(E_{C}^{GGA} - E_{C}^{LDA}),
\end{equation}

where $E_{C}^{GGA}$ is a correlation energy obtained within the GGA framework, and $E_{X}^{GGA}$ is the exchange energy also within GGA. The three parameters (hence the 3 in B3LYP) are set to $a_0=0.2, a_X=0.72$, and $a_C=0.81$.

We obviously need to modify the input file accordingly. Move back into the parent directory and make a new folder there named `B3LYP`. Within the new `B3LYP` folder, make a new subfolder `SZV`, in which we will perform B3LYP calculations using the SZV basis set:

```bash
$ cd ../..
$ mkdir B3LYP
$ cd B3LYP
$ mkdir SZV
$ cd SZV
```

Just as for the PBE case, copy the `H2O_template.inp` file from the parent directory into the current one and rename it:

```bash
$ cp ../../H2O_template.inp H2O.inp
```

Open the `H2O.inp` file using vim. We should replace the entire `&XC` section with:

```
 &XC
   &XC_FUNCTIONAL
     &LYP
       SCALE_C 0.81 ! mixing parameters explained above
     &END
     &BECKE88
       SCALE_X 0.72
     &END
     &XALPHA
       SCALE_X 0.08
     &END
   &END XC_FUNCTIONAL
   &HF
     &SCREENING
       EPS_SCHWARZ 1.0E-10
     &END
     &MEMORY
       MAX_MEMORY 5
     &END
     FRACTION 0.20
   &END
 &END XC
```

but it is possible to use the @INCLUDE preprocessing directive, to include the file **b3lyp.inc** instead of **pbe.inc** 
Check the contents of the two include files in the main directory.

#### Assignment 3: The simulations for the SZV basis set: B3LYP

Save and close the `H2O.inp` file, and finally submit the calculation.

Then repeat the above procedures and and submit the calculations for `ethanol.inp`, `ethene.inp`, and `ts.inp` for the B3LYP hybrid functional. 

In [ ]:
## -- Your code here

#### End Assignment 3

# 3. An excursus on basis sets in CP2K and an analysis of our results

A terse introduction to the framework of basis sets in CP2K can be found [here](https://www.cp2k.org/basis_sets). The basis sets that we have / we will examine in this exercise are, in increasing order of complexity:

* SZV-MOLOPT-GTH
* DZVP-MOLOPT-GTH
* TZVP-MOLOPT-GTH
* TZV2P-MOLOPT-GTH
* TZV2PX-MOLOPT-GTH

where SZ, DZ, and TZ stand for single-, double-, and triple-zeta, respectively, and refer to how many functions are used to describe each atomic orbital: in this sense, it gives an idea as to the flexibility of each basis set. For instance, in TZ, each primitive vector is treated as a full-fledged basis function, which is better in regards to accuracy, yet is (of course) more expensive computationally. `V` stands for valence: here, it means that only valence orbitals are split, whereas core orbitals are not. `P` stands for polarized, and means that extra functions are included in order to include the polarization of the electron density. The label `MOLOPT` indicates that these sets are optimized to perform accurate molecular calculations. In this library the larger basis sets are selected as extensions of smaller ones.

Following the spirit of [Lopes *et. al.* (clickable link)](https://pubs.rsc.org/en/content/articlelanding/2016/RA/C6RA08509A#!divAbstract), we want to perform the above calculations using both functionals PBE and B3LYP using a total of two or three of the above basis sets, and finally compare the results with those of the paper.

#### Assignment 4: Simulations for both PBE and B3LYP functionals using other basis sets
Repeat the simulations for the `ENERGY` run of `ethanol`, `ethene`, and `H2O`, as well as for the transition state `ts`, using the `DZVP-MOLOPT-GTH` and `TZVP-MOLOPT-GTH` basis sets. The **only** thing you'll need to change in the respective input files is the value of the `BASIS_SET` keyword under the `&KIND` subsection:

```
&KIND H
   BASIS_SET DZVP-MOLOPT-GTH
   POTENTIAL GTH-PBE-q1
 &END KIND
```
 
Since we should change this for all atom species in the input file, we used a trick:
**we defined a variable at the beginning, and then use it for all species. You simply need to change the value of the variable!**
Notice that we leave the value of the `POTENTIAL` keyword the same! This is because our treatment using pseudopotentials does not change from functional to functional or basis set to basis set. 

In [ ]:
## -- Your code here

#### End Assignment 4

#### Assignment 5: Reproduction of the energy profiles for functionals and basis sets
Reproduce the energy profiles of Fig. 4 in [the paper](https://pubs.rsc.org/en/content/articlepdf/2016/ra/c6ra08509a) by plotting the absolute energies
vs. the reaction species for each of your simulations. The converged energy is just the final value at the bottom of the respective output file, and can be obtained with the command: 

    grep "ENERGY|" ethanol.out
    
this will give a string that can be processed with our jupyter. 

You’ll have six different
curves (2 functionals PBE, B3LYP × 3 basis sets SZV, DZVP, TZVP), so you
can, depending on how you scale the axes, just superimpose them onto one graph.
**Important!: don’t forget that CP2K output files are in atomic units, so
a unit conversion from e.g., Ha energies to eV might be sensible.**

In [ ]:
## -- Your code here

#### End Assignment 5

# THE FOLLOWING IS FOR THE LECTURE IN MAY ABOUT FREE ENERGIES

#### Assignment May-6: Reproduction of the rate constant for the reaction $k(T)$
Thermodynamics defines the temperature-dependent **rate constant** $k(T)$ as

$$k(T)=\frac{k_BT}{2\pi\hbar C}\exp(-\Delta G / RT),$$

where $k_B$ is the Boltzmann constant, $T$ the absolute temperature, $\hbar$ the reduced Planck constant, $C$ the standard concentration of 1 mol $L^{-1}$, $G$ the **Gibbs free energy**, and $R$ the universal gas constant. The **Gibbs free energy** is in turn defined in terms of the **enthalpy** $H=U+pV$ as

$$ G=H-TS, $$

where $U$ is the **internal energy** (which we obtain from our converged calculations), $p$ the pressure, $V$ the volume, and $S$ the entropy. By considering only *differences* in the various energies, we posit
$$\Delta G = \Delta H = \Delta U. $$

The equation for $k(T)$ then becomes
$$ k(T)=\frac{k_BT}{2\pi\hbar C}\exp(-\Delta U / RT), $$

where $\Delta U := U_{\text{final}} - U_{\text{init}}$ is obtained from our calculations. Consistent with the literature (and with the notation in Fig. 2), anything having to do with the **transition state** takes a superscript double-dagger: thus, $U^\ddagger$, $G^\ddagger$, etc. Reproduce the gasphase plot (black squares) of Fig. 6 [the paper](https://pubs.rsc.org/en/content/articlepdf/2016/ra/c6ra08509a) by plotting $\log[k^\ddagger(T)]$ vs. $T$, with

$$k^\ddagger(T)=\frac{k_BT}{2\pi\hbar C}\exp(-\Delta U^\ddagger / RT),$$

and using the identification that $U^\ddagger=U_{\text{TS}} - U_{\text{ethanol}}$ for about 20 evenly-spaced values of $T$ between 300K and 1300K. You'll have six different curves (2 functionals PBE, B3LYP x 3 basis sets SZV, DZVP, TZVP), so you can, depending on how you scale the axes, just superimpose them onto one graph.

In [ ]:
## -- Your code here

#### End Assignment May-6